# Notebook Huấn luyện Model Nhận dạng Biển số xe

## Chuẩn bị: Kích hoạt GPU
Trước khi chạy bất cứ thứ gì, bạn cần đảm bảo đã bật GPU cho notebook này.
1. Trên thanh menu, chọn **Runtime**.
2. Chọn **Change runtime type**.
3. Trong mục `Hardware accelerator`, chọn **GPU**.
4. Trong mục `GPU type`, chọn **T4**.
5. Nhấn **Save**.

Cell code ngay bên dưới sẽ giúp bạn kiểm tra lại thông tin GPU đã nhận được.

In [ ]:
!nvidia-smi

## 1. Cài đặt Môi trường và Thư viện
Bước này sẽ cài đặt các thư viện cần thiết như `ultralytics` cho YOLO và các thư viện phụ trợ khác.

In [ ]:
%pip install ultralytics opencv-python pyyaml

## 2. Kết nối Google Drive
Chúng ta sẽ kết nối với Google Drive để có thể lưu trữ dataset, các file cấu hình, và trọng số (weights) của model đã huấn luyện.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3. Chuẩn bị Dataset từ file Zip
Copy file .zip từ Google Drive sang máy ảo Colab và giải nén. Việc này giúp tăng tốc độ đọc ghi dữ liệu trong quá trình training.

In [ ]:
import os

os.makedirs('/content/license_plate_dataset', exist_ok=True)

!cp /content/drive/MyDrive/projects/license_plate_detection_recognition/training/dataset/LP_detection.zip /content/LP_detection.zip
!cp /content/drive/MyDrive/projects/license_plate_detection_recognition/training/dataset/OCR.zip /content/OCR.zip

!unzip -q /content/LP_detection.zip -d /content/license_plate_dataset/
!unzip -q /content/OCR.zip -d /content/license_plate_dataset/

## 4. Tạo file cấu hình `data.yaml`
Tạo file `data.yaml` tự động để trỏ đến các thư mục vừa giải nén trên máy ảo Colab.

In [ ]:
import yaml

data_yaml = {
    'train': '/content/license_plate_dataset/LP_detection/images/train',
    'val': '/content/license_plate_dataset/LP_detection/images/val',
    'nc': 1,
    'names': ['license_plate']
}

with open('/content/data.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=False)

%cat /content/data.yaml

## 5. Huấn luyện Model YOLO (Detection)
Bắt đầu quá trình training với các tham số đã được tối ưu cho GPU T4.

In [ ]:
from ultralytics import YOLO

# Load a pre-trained model (recommended) or a new model
model = YOLO('yolo11n.pt')

# Train the model
results = model.train(
   data='/content/data.yaml',
   epochs=100,
   imgsz=640,
   project='/content/drive/MyDrive/model_license_plate_runs',
   name='exp'
)

## 6. Đánh giá và Sử dụng Model
Sau khi training, bạn có thể đánh giá model trên tập validation và sử dụng nó để nhận diện trên ảnh mới.

In [ ]:
# Tải model tốt nhất đã được huấn luyện
best_model = YOLO('/content/drive/MyDrive/model_license_plate_runs/exp5/weights/last.pt')

# Test nhận diện trên một ảnh mới
results = best_model('path/to/your/test_image.jpg')

# Process results
for r in results:
    boxes = r.boxes
    r.show()